In [1]:
# Neural network BERT-powered text classification

# Dependencies
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import os
import toml
import re
import itertools
from text_cleaner import *
import operator
from collections import Counter
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

def extract_hashtags(post_text):
    HASH_RE = re.compile(r"\#\w+")
    out_list = re.findall(HASH_RE, post_text)
    return out_list

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Load the .csv with archetypes
arch_df = pd.read_csv('archetypes_pl.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [3]:
# Table preprocessing - replace all NaN with 2 (Unrelated/Don't know class), replace 0-5 values with the ones in range -1.0 - 1.0
arch_df = arch_df.fillna(2.0)

# Remove duplicated annotations, to exclude conflicting entries
arch_df = arch_df[~arch_df.index.duplicated(keep='first')]

# Print the head of the dataset after modification
arch_df.head()

,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [21]:
# Create pandas-like dataset
BASE_DIR = "instagram_cleared"

def clean_post(src_text: str) -> str:
    # Extract posts and hashtags
    extracted_text = remove_stopwords(clean_up_text(src_text))
    extracted_hashtags = extract_hashtags(src_text)
    return extracted_text + extracted_hashtags

def generate_dataset(dataset_df: pd.DataFrame) -> pd.DataFrame:
    out_dataset = pd.DataFrame(columns=["text"] + list(dataset_df.columns))
    # Iterate over whole DataFrame
    for i, row in tqdm(dataset_df.iterrows()):
        trait_row = copy.deepcopy(row)
        profile_posts = []

        # Get all posts per profile
        profile_path = os.path.join(BASE_DIR, i)
        for file in os.listdir(profile_path):
            if not file.endswith(".toml"):
                with open(os.path.join(profile_path, file), "r") as post_f:
                    read_text = post_f.read()
                    profile_posts.extend(clean_post(read_text))
        trait_row["text"] = " ".join(profile_posts)
        out_dataset = out_dataset.append(trait_row)
    out_dataset = out_dataset.reset_index(drop=True)
    return out_dataset

# Initialize text dataset DataFrames
full_dataset = generate_dataset(arch_df)

508it [00:15, 33.67it/s]


In [69]:
# Model creation and training
dataset_copy = copy.deepcopy(full_dataset)

def format_labels(data_set):
    return (tf.keras.utils.to_categorical(np.array(data_set.pop("innocent"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("sage"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("explorer"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("outlaw"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("magician"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("hero"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("lover"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("jester"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("everyman"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("caregiver"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("ruler"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("creator"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("dominant"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("submissive"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("maximalist"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("minimalist"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("inspiring"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("systematic"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("discovering"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("conservative"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("verifying"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("overlooking"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("sharpening"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("harmonic"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("empathic"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("matter_of_fact"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("brave"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("protective"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("generous"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("thrifty"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("favourable"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("balanced"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("sensuality"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("intelligent"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("believe"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("egocentric"))),
    tf.keras.utils.to_categorical(np.array(data_set.pop("allocentric"))))

X = dataset_copy.pop("text")
y = np.array(format_labels(dataset_copy))
print(y.shape)
y = np.transpose(y, (1, 0, 2))

(37, 508, 5)


In [70]:
print(y.shape)

(508, 37, 5)


In [71]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [72]:
print(X_test.shape)
print(y_test.shape)

(102,)
(102, 37, 5)


In [73]:
# Text encoder
VOCAB_CNT = 160000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_CNT)
encoder.adapt(np.array(X_train))

In [74]:
# Model - Functional API
def build_model(embedded_dim: int) -> tf.keras.Model:
    def encode_text(inputs):
        return encoder(inputs)
    
    def softmax_layer(inputs, layer_name: str):
        return 
    
    # Define model base - Embedding and LSTMs
    inputs = tf.keras.Input(shape=(None, ), dtype=tf.string, name="text_input")
    encoded_input = encode_text(inputs)
    x = tf.keras.layers.Embedding(len(encoder.get_vocabulary()), embedded_dim, mask_zero=True)(encoded_input)
    x = tf.keras.layers.SpatialDropout1D(0.2)(x)
    x = tf.keras.layers.LSTM(embedded_dim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    # Output layers
    # Archetypes
    innocent_out = tf.keras.layers.Dense(5, name="innocent_out", activation="softmax")(x)
    sage_out = tf.keras.layers.Dense(5, name="sage_out", activation="softmax")(x)
    explorer_out = tf.keras.layers.Dense(5, name="explorer_out", activation="softmax")(x)
    outlaw_out = tf.keras.layers.Dense(5, name="outlaw_out", activation="softmax")(x)
    magician_out = tf.keras.layers.Dense(5, name="magician_out", activation="softmax")(x)
    hero_out = tf.keras.layers.Dense(5, name="hero_out", activation="softmax")(x)
    lover_out = tf.keras.layers.Dense(5, name="lover_out", activation="softmax")(x)
    jester_out = tf.keras.layers.Dense(5, name="jester_out", activation="softmax")(x)
    everyman_out = tf.keras.layers.Dense(5, name="everyman_out", activation="softmax")(x)
    caregiver_out = tf.keras.layers.Dense(5, name="caregiver_out", activation="softmax")(x)
    ruler_out = tf.keras.layers.Dense(5, name="ruler_out", activation="softmax")(x)
    creator_out = tf.keras.layers.Dense(5, name="creator_out", activation="softmax")(x)
    # Traits
    dominant_out = tf.keras.layers.Dense(5, name="dominant_out", activation="softmax")(x)
    submissive_out = tf.keras.layers.Dense(5, name="submissive_out", activation="softmax")(x)
    maximalist_out = tf.keras.layers.Dense(5, name="maximalist_out", activation="softmax")(x)
    minimalist_out = tf.keras.layers.Dense(5, name="minimalist_out", activation="softmax")(x)
    inspiring_out = tf.keras.layers.Dense(5, name="inspiring_out", activation="softmax")(x)
    systematic_out = tf.keras.layers.Dense(5, name="systematic_out", activation="softmax")(x)
    discovering_out = tf.keras.layers.Dense(5, name="discovering_out", activation="softmax")(x)
    conservative_out = tf.keras.layers.Dense(5, name="conservative_out", activation="softmax")(x)
    verifying_out = tf.keras.layers.Dense(5, name="verifying_out", activation="softmax")(x)
    overlooking_out = tf.keras.layers.Dense(5, name="overlooking_out", activation="softmax")(x)
    sharpening_out = tf.keras.layers.Dense(5, name="sharpening_out", activation="softmax")(x)
    harmonic_out = tf.keras.layers.Dense(5, name="harmonic_out", activation="softmax")(x)
    empathic_out = tf.keras.layers.Dense(5, name="empathic_out", activation="softmax")(x)
    matter_of_fact_out = tf.keras.layers.Dense(5, name="matter_of_fact_out", activation="softmax")(x)
    brave_out = tf.keras.layers.Dense(5, name="brave_out", activation="softmax")(x)
    protective_out = tf.keras.layers.Dense(5, name="protective_out", activation="softmax")(x)
    generous_out = tf.keras.layers.Dense(5, name="generous_out", activation="softmax")(x)
    thrifty_out = tf.keras.layers.Dense(5, name="thrifty_out", activation="softmax")(x)
    favourable_out = tf.keras.layers.Dense(5, name="favourable_out", activation="softmax")(x)
    balanced_out = tf.keras.layers.Dense(5, name="balanced_out", activation="softmax")(x)
    sensuality_out = tf.keras.layers.Dense(5, name="sensuality_out", activation="softmax")(x)
    intelligent_out = tf.keras.layers.Dense(5, name="intelligent_out", activation="softmax")(x)
    believe_out = tf.keras.layers.Dense(5, name="believe_out", activation="softmax")(x)
    egocentric_out = tf.keras.layers.Dense(5, name="egocentric_out", activation="softmax")(x)
    allocentric_out = tf.keras.layers.Dense(5, name="allocentric_out", activation="softmax")(x)
    
    # Define full model
    model = tf.keras.Model(
        inputs=inputs,
        outputs=[
            innocent_out,
            sage_out,
            explorer_out,
            outlaw_out,
            magician_out,
            hero_out,
            lover_out,
            jester_out,
            everyman_out,
            caregiver_out,
            ruler_out,
            creator_out,
            dominant_out,
            submissive_out,
            maximalist_out,
            minimalist_out,
            inspiring_out,
            systematic_out,
            discovering_out,
            conservative_out,
            verifying_out,
            overlooking_out,
            sharpening_out,
            harmonic_out,
            empathic_out,
            matter_of_fact_out,
            brave_out,
            protective_out,
            generous_out,
            thrifty_out,
            favourable_out,
            balanced_out,
            sensuality_out,
            intelligent_out,
            believe_out,
            egocentric_out,
            allocentric_out
        ])
    
    return model

In [75]:
# Training callbacks
model = build_model(embedded_dim=128)

# Generate output list
output_list = "val_innocent_out_accuracy,val_sage_out_accuracy,val_explorer_out_accuracy,val_outlaw_out_accuracy,val_magician_out_accuracy,val_hero_out_accuracy,val_lover_out_accuracy,val_jester_out_accuracy,val_everyman_out_accuracy,val_caregiver_out_accuracy,val_ruler_out_accuracy,val_creator_out_accuracy,val_dominant_out_accuracy,val_submissive_out_accuracy,val_maximalist_out_accuracy,val_minimalist_out_accuracy,val_inspiring_out_accuracy,val_systematic_out_accuracy,val_discovering_out_accuracy,val_conservative_out_accuracy,val_verifying_out_accuracy,val_overlooking_out_accuracy,val_sharpening_out_accuracy,val_harmonic_out_accuracy,val_empathic_out_accuracy,val_matter_of_fact_out_accuracy,val_brave_out_accuracy,val_protective_out_accuracy,val_generous_out_accuracy,val_thrifty_out_accuracy,val_favourable_out_accuracy,val_balanced_out_accuracy,val_sensuality_out_accuracy,val_intelligent_out_accuracy,val_believe_out_accuracy,val_egocentric_out_accuracy,val_allocentric_out_accuracy".split(",")

# Plot graph
tf.keras.utils.plot_model(model, "my_model.png", show_shapes=True)

# Compile the model
# Compile the model
model.compile(loss={
                    "innocent_out": tf.keras.losses.CategoricalCrossentropy(),
                    "sage_out": tf.keras.losses.CategoricalCrossentropy(),
                    "explorer_out": tf.keras.losses.CategoricalCrossentropy(),
                    "outlaw_out": tf.keras.losses.CategoricalCrossentropy(),
                    "magician_out": tf.keras.losses.CategoricalCrossentropy(),
                    "hero_out": tf.keras.losses.CategoricalCrossentropy(),
                    "lover_out": tf.keras.losses.CategoricalCrossentropy(),
                    "jester_out": tf.keras.losses.CategoricalCrossentropy(),
                    "everyman_out": tf.keras.losses.CategoricalCrossentropy(),
                    "caregiver_out": tf.keras.losses.CategoricalCrossentropy(),
                    "ruler_out": tf.keras.losses.CategoricalCrossentropy(),
                    "creator_out": tf.keras.losses.CategoricalCrossentropy(),
                    "dominant_out": tf.keras.losses.CategoricalCrossentropy(),
                    "submissive_out": tf.keras.losses.CategoricalCrossentropy(),
                    "maximalist_out": tf.keras.losses.CategoricalCrossentropy(),
                    "minimalist_out": tf.keras.losses.CategoricalCrossentropy(),
                    "inspiring_out": tf.keras.losses.CategoricalCrossentropy(),
                    "systematic_out": tf.keras.losses.CategoricalCrossentropy(),
                    "discovering_out": tf.keras.losses.CategoricalCrossentropy(),
                    "conservative_out": tf.keras.losses.CategoricalCrossentropy(),
                    "verifying_out": tf.keras.losses.CategoricalCrossentropy(),
                    "overlooking_out": tf.keras.losses.CategoricalCrossentropy(),
                    "sharpening_out": tf.keras.losses.CategoricalCrossentropy(),
                    "harmonic_out": tf.keras.losses.CategoricalCrossentropy(),
                    "empathic_out": tf.keras.losses.CategoricalCrossentropy(),
                    "matter_of_fact_out": tf.keras.losses.CategoricalCrossentropy(),
                    "brave_out": tf.keras.losses.CategoricalCrossentropy(),
                    "protective_out": tf.keras.losses.CategoricalCrossentropy(),
                    "generous_out": tf.keras.losses.CategoricalCrossentropy(),
                    "thrifty_out": tf.keras.losses.CategoricalCrossentropy(),
                    "favourable_out": tf.keras.losses.CategoricalCrossentropy(),
                    "balanced_out": tf.keras.losses.CategoricalCrossentropy(),
                    "sensuality_out": tf.keras.losses.CategoricalCrossentropy(),
                    "intelligent_out": tf.keras.losses.CategoricalCrossentropy(),
                    "believe_out": tf.keras.losses.CategoricalCrossentropy(),
                    "egocentric_out": tf.keras.losses.CategoricalCrossentropy(),
                    "allocentric_out": tf.keras.losses.CategoricalCrossentropy()
                },
              optimizer=tf.keras.optimizers.RMSprop(1e-3),
              loss_weights={
                    "innocent_out": 1.0,
                    "sage_out": 1.0,
                    "explorer_out": 1.0,
                    "outlaw_out": 1.0,
                    "magician_out": 1.0,
                    "hero_out": 1.0,
                    "lover_out": 1.0,
                    "jester_out": 1.0,
                    "everyman_out": 1.0,
                    "caregiver_out": 1.0,
                    "ruler_out": 1.0,
                    "creator_out": 1.0,
                    "dominant_out": 1.0,
                    "submissive_out": 1.0,
                    "maximalist_out": 1.0,
                    "minimalist_out": 1.0,
                    "inspiring_out": 1.0,
                    "systematic_out": 1.0,
                    "discovering_out": 1.0,
                    "conservative_out": 1.0,
                    "verifying_out": 1.0,
                    "overlooking_out": 1.0,
                    "sharpening_out": 1.0,
                    "harmonic_out": 1.0,
                    "empathic_out": 1.0,
                    "matter_of_fact_out": 1.0,
                    "brave_out": 1.0,
                    "protective_out": 1.0,
                    "generous_out": 1.0,
                    "thrifty_out": 1.0,
                    "favourable_out": 1.0,
                    "balanced_out": 1.0,
                    "sensuality_out": 1.0,
                    "intelligent_out": 1.0,
                    "believe_out": 1.0,
                    "egocentric_out": 1.0,
                    "allocentric_out": 1.0
                },
              metrics={
                    "innocent_out": "accuracy",
                    "sage_out": "accuracy",
                    "explorer_out": "accuracy",
                    "outlaw_out": "accuracy",
                    "magician_out": "accuracy",
                    "hero_out": "accuracy",
                    "lover_out": "accuracy",
                    "jester_out": "accuracy",
                    "everyman_out": "accuracy",
                    "caregiver_out": "accuracy",
                    "ruler_out": "accuracy",
                    "creator_out": "accuracy",
                    "dominant_out": "accuracy",
                    "submissive_out": "accuracy",
                    "maximalist_out": "accuracy",
                    "minimalist_out": "accuracy",
                    "inspiring_out": "accuracy",
                    "systematic_out": "accuracy",
                    "discovering_out": "accuracy",
                    "conservative_out": "accuracy",
                    "verifying_out": "accuracy",
                    "overlooking_out": "accuracy",
                    "sharpening_out": "accuracy",
                    "harmonic_out": "accuracy",
                    "empathic_out": "accuracy",
                    "matter_of_fact_out": "accuracy",
                    "brave_out": "accuracy",
                    "protective_out": "accuracy",
                    "generous_out": "accuracy",
                    "thrifty_out": "accuracy",
                    "favourable_out": "accuracy",
                    "balanced_out": "accuracy",
                    "sensuality_out": "accuracy",
                    "intelligent_out": "accuracy",
                    "believe_out": "accuracy",
                    "egocentric_out": "accuracy",
                    "allocentric_out": "accuracy"
                })

early_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_y_accuracy', min_delta=0, patience=10, verbose=1,
    mode='auto', baseline=None, restore_best_weights=False
)

class ModdedModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
    def on_epoch_end(self, epoch, logs={}):
        relevantAcc = output_list
        accuracies = [logs[k] for k in relevantAcc]
        average = sum(accuracies) / len(relevantAcc)
        print(f"Average Accuracies: {average}")
        logs["val_y_accuracy"] = average
        super(tf.keras.callbacks.ModelCheckpoint, self).on_epoch_end(
            epoch, logs=logs
        )
        
mmc_callback = ModdedModelCheckpoint(
    filepath="train_test_instagram/checkpoint_total",
    monitor="val_y_accuracy",
    save_weights_only=True,
    mode='max',
    save_best_only=True,
    verbose=1
)
    
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs_single_model")

with tf.device("/GPU:0"):
    history = model.fit(X_train, y_train, epochs=100,
                        batch_size=8,
                        validation_data=(X_test, y_test),
                        verbose=1,
                        callbacks=[mmc_callback,
                                   tensorboard_callback,
                                   early_callback])

Epoch 1/100


ValueError: in user code:

    /mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /mnt/HDD_Linux/Praca_magisterska/instagram_analysis/insta_venv/lib/py

In [207]:
# Evaluate results
test_results = model.evaluate(X_test, y_test)

print('Test results:', test_results)

4/4 [==============================] - 1s 278ms/step - loss: 54.2222 - innocent_out_loss: 1.3254 - sage_out_loss: 1.4108 - explorer_out_loss: 1.6772 - outlaw_out_loss: 1.4322 - magician_out_loss: 1.3550 - hero_out_loss: 1.5306 - lover_out_loss: 1.5714 - jester_out_loss: 1.5044 - everyman_out_loss: 1.3672 - caregiver_out_loss: 1.4501 - ruler_out_loss: 1.5217 - creator_out_loss: 1.4723 - dominant_out_loss: 1.5849 - submissive_out_loss: 1.5447 - maximalist_out_loss: 1.4040 - minimalist_out_loss: 1.5429 - inspiring_out_loss: 1.2766 - systematic_out_loss: 1.5149 - discovering_out_loss: 1.3645 - conservative_out_loss: 1.5214 - verifying_out_loss: 1.3918 - overlooking_out_loss: 1.5201 - sharpening_out_loss: 1.6247 - harmonic_out_loss: 1.2363 - empathic_out_loss: 1.2662 - matter_of_fact_out_loss: 1.5342 - brave_out_loss: 1.5687 - protective_out_loss: 1.5824 - generous_out_loss: 1.2133 - thrifty_out_loss: 1.5309 - favourable_out_loss: 1.6203 - balanced_out_loss: 1.2727 - sensuality_out_loss: 1.

In [200]:
print(model.predict(np.array([" ".join(clean_post("""Czy można jeść słodycze podczas odchudzania?🍭⁣⁣
⁣⁣
Cześć!:)⁣⁣
⁣⁣
Często można zauważyć podejście u osób, które się odchudzają, że odrzucają oni w 100% cukier albo propagują oni, że jednym słusznym podejściem jest eliminacja wszystkich przyjemności. 🍩⁣⁣
⁣⁣
Z drugiej strony, czasami widzimy osoby, które propagują, że można spożywać wszystko, tylko wystarczy liczyć kalorię i to schudniemy.🍫 ⁣⁣
⁣⁣
📍 Natomiast dzisiaj, dzielę się z Tobą moim sposobem, który u mnie działa oraz u moich podopiecznych. Podchodzę do tego w taki sposób, że większość mojego jadłospisu, jest bazowanie na zdrowych produktach, natomiast, jeżeli, uzupełnimy wszystko, co potrzebne jest dla naszego ciała do prawidłowego funkcjonowania, to wtedy wdrażam, te produkty rekreacyjne, które sprawiają mi przyjemność. ⁣⁣
⁣⁣
🤓 Uważam, że jest to ważne, żeby mieć satysfakcję z naszej diety, bo to pozwoli mi, wytrwać na niej o wiele dłużej, niż jakbym miał jakieś restrykcje.⁣⁣
⁣⁣
Oczywiście, jest to schemat, który nie pasuję dla każdego.⁣⁣
⁣⁣
I zawsze, gdy ktoś pyta się mnie, czy można jeść słodycze, zadaję pewne pytania.⁣⁣
⁣⁣
➡️ Czy mieści się to w ujemnym bilansie kalorycznym?⁣⁣
➡️ Czy nawet jak się mieści, spełniasz całe zapotrzebowanie na witaminy, błonnik czy składniki mineralne?⁣⁣
➡️ Czy po zjedzeniu tej słodkości, nie sięgniesz po więcej i przez to przekroczysz swoje zapotrzebowanie?⁣⁣
⁣⁣
⁣⁣
🎥 Nagrałem o tym materiał, w którym, właśnie szerzej omawiam ten temat, także zachęcam sprawdzić.⁣⁣
Link znajdziesz w bio!⁣⁣
⁣⁣
Pozdrawiam i życzę miłego dnia! ❤️⁣⁣
⁣⁣
PS. Jestem ciekaw, czy jeżeli, mógł/a byś wybrać jakiś produkt, to byłby na słodko, czy słono?:)🍕🍦⁣⁣
⁣⁣
#śniadanie #zdrowejedzenie #fit #kawa #goodmorning #wiemcojem #jedzenie #sniadanie #yummy #dieta #poland #zdrowo #coffee #dziendobry #delicious #morning #sniadaniemistrzow #instagood #foodie #warsaw #odchudzanie #owsianka #breakfasttime #eggs #czystamicha #tasty #obiad #healthybreakfast #redukcja
"""
))])))

[array([[0.01137773, 0.03182445, 0.37314755, 0.5130092 , 0.07064112]],
      dtype=float32), array([[0.03023164, 0.06084527, 0.65663975, 0.1874094 , 0.06487393]],
      dtype=float32), array([[0.02506514, 0.04129549, 0.63308275, 0.19384618, 0.10671043]],
      dtype=float32), array([[0.033358  , 0.11437706, 0.7786283 , 0.05397646, 0.01966022]],
      dtype=float32), array([[0.0140657 , 0.04916009, 0.80596477, 0.11364656, 0.01716293]],
      dtype=float32), array([[0.02278624, 0.08337032, 0.7900975 , 0.07307991, 0.03066601]],
      dtype=float32), array([[0.0310161 , 0.03432339, 0.5122559 , 0.35689077, 0.06551388]],
      dtype=float32), array([[0.02299303, 0.07274623, 0.7047842 , 0.14514391, 0.05433256]],
      dtype=float32), array([[0.00821524, 0.0242739 , 0.32810032, 0.37211043, 0.26730013]],
      dtype=float32), array([[0.01569674, 0.06092544, 0.544902  , 0.21497466, 0.16350119]],
      dtype=float32), array([[0.02093195, 0.0532625 , 0.7632376 , 0.10182117, 0.06074682]],
      dty